In [1]:
using Pkg
Pkg.activate(".")
using Statistics
using AdversarialAttacks
using BSON: @save, @load
using Images
using Plots
using Flux
using MLDatasets

  Activating project at `~/Documents/WS25/Julia/AdversarialAttacks.jl`


In [2]:
train_model = true

true

In [ ]:
c10_train = MLDatasets.CIFAR10(:train)
c10_test = MLDatasets.CIFAR10(:test)

In [ ]:
function preprocess(dataset)
    x, y = dataset[:]

    x = reshape(x, 32, 32, 3, :)

    # One-hot encode targets
    y = Flux.onehotbatch(y, 0:9)

    return x, y
end

In [ ]:
x_train, y_train = preprocess(c10_train)
x_test, y_test = preprocess(c10_test)

In [ ]:
losses = Float32[]
if train_model
    println("Training model...")
    batchsize = 256
    train_loader = Flux.DataLoader((x_train, y_train); batchsize=batchsize, shuffle=true);
    model = Flux.Chain(
        Flux.Conv((3, 3), 3 => 32, pad=1, Flux.relu),
        Flux.Conv((3, 3), 32 => 32, pad=1, Flux.relu),
        Flux.MaxPool((2, 2)),

        Flux.Conv((3, 3), 32 => 64, pad=1, Flux.relu),
        Flux.Conv((3, 3), 64 => 64, pad=1, Flux.relu),
        Flux.MaxPool((2, 2)),

        Flux.flatten,
        Flux.Dense(8 * 8 * 64, 256, Flux.relu),
        Flux.Dense(256, 10),
    )
    loss_fn(ŷ, y) = Flux.logitcrossentropy(ŷ, y)
    optim = Flux.setup(Flux.Adam(0.001), model)

    for epoch in 1:5
        for (i, (x, y)) in enumerate(train_loader)
            loss, grads = Flux.withgradient(m -> loss_fn(m(x), y), model)
            Flux.update!(optim, model, grads[1])
            push!(losses, loss)
            if isone(i) || iszero(i % 50)
                acc = accuracy(model) * 100
                println("Epoch $epoch, step $i:\t loss = $(loss), acc = $(acc)%")
            end
        end
    end

    # Save the trained model
    @save "trained_model.bson" model
else
    # Reload the model from file
    @load "trained_model.bson" model
    println("Model loaded successfully!")
end

In [ ]:
function accuracy(model)
    # Use onecold to return class index
    ŷ = Flux.onecold(model(x_test))
    y = Flux.onecold(y_test)

    return mean(ŷ .== y)
end

In [ ]:
if train_model
    plot(losses, title="Training Loss over Time", xlabel="Iteration", ylabel="Loss", legend=false)
end

In [ ]:
# Wrap the Flux model
flux_model = FluxModel(model)

# Select a single image (index 9) and keep batch dimension for the model
image_idx = 164
x_sample = Float32.(x_test[:, :, :, image_idx:image_idx])  # Shape: 32×32×3×1, convert to Float32
y_sample = y_test[:, image_idx]  # One-hot vector for this image

# Create FGSM attack 
fgsm = FGSM(Dict("epsilon" => 0.05))

# Generate adversarial example
sample = (data=x_sample, label=y_sample)
adv_sample = attack(fgsm, flux_model, sample)

In [ ]:
# Get predictions
original_pred = model(x_sample)
adv_pred = model(adv_sample)

# Convert to class labels
original_class = Flux.onecold(original_pred, 0:9)[1]
adv_class = Flux.onecold(adv_pred, 0:9)[1]
true_class = Flux.onecold(y_sample, 0:9)

# CIFAR-10 class names
class_names = ["plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

# Extract single images (remove batch dimension)
img_original = x_sample[:, :, :, 1]  # 32x32x3
img_adv = adv_sample[:, :, :, 1]     # 32x32x3

# Clamp to valid range [0, 1]
img_original = clamp.(img_original, 0f0, 1f0)
img_adv = clamp.(img_adv, 0f0, 1f0)

# Convert to RGB format for display (CHW to CWH for colorview)
img_orig_rgb = colorview(RGB, permutedims(img_original, (3, 2, 1)))
img_adv_rgb = colorview(RGB, permutedims(img_adv, (3, 2, 1)))

# Plot side by side
p1 = plot(img_orig_rgb, 
          title="Original\nTrue: $(class_names[true_class+1])\nPred: $(class_names[original_class+1])")
p2 = plot(img_adv_rgb,
          title="Adversarial (ε=$(fgsm.parameters["epsilon"]))\nTrue: $(class_names[true_class+1])\nPred: $(class_names[adv_class+1])")

plot(p1, p2)